In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\suban\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
pip install pandas pymysql sqlalchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\suban\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\suban\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import csv
import mysql.connector
from mysql.connector import Error

In [4]:
# --- Configuration ---
CSV_FILE = 'Bookings-100000-Rows.csv'
DATABASE_NAME = 'ola'
TABLE_NAME = 'booking'

# !! IMPORTANT !!
# Replace with your MySQL database credentials
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Subanisyed7181@'
}

In [7]:
from datetime import datetime

In [ ]:
def insert_data_from_csv(csv_file_path, db_name, table_name, db_config):
    """
    Inserts data from a CSV file into a MySQL database table.

    Args:
        csv_file_path (str): The path to the CSV file.
        db_name (str): The name of the database.
        table_name (str): The name of the table.
        db_config (dict): A dictionary containing database connection details
                          (host, user, password).
    """
    conn = None
    cursor = None
    try:
        
        conn = mysql.connector.connect(
            host=db_config['host'],
            user=db_config['user'],
            password=db_config['password'],
            database=db_name 
        )
        if conn.is_connected():
            print(f"Successfully connected to MySQL Database '{db_name}'!")

            cursor = conn.cursor()

            
            columns = [
                "booking_date", "booking_time", "booking_id", "booking_status",
                "customer_id", "vehicle_type", "pickup_location", "drop_location",
                "v_tat", "c_tat", "canceled_rides_by_customer",
                "canceled_rides_by_driver", "incomplete_rides",
                "incomplete_rides_reason", "booking_value", "payment_method",
                "ride_distance", "driver_ratings", "customer_rating"
            ]

            
            placeholders = ', '.join(['%s'] * len(columns))
            insert_columns_str = ', '.join([f"`{col}`" for col in columns])
            insert_query = f"INSERT INTO `{table_name}` ({insert_columns_str}) VALUES ({placeholders})"

            print(f"Prepared INSERT query for table '{table_name}'.")

            
            with open(csv_file_path, 'r', encoding='utf-8') as file:
                csv_reader = csv.reader(file)
                next(csv_reader) 
                batch_size = 10000 
                data_to_insert = []
                total_rows_processed = 0

                for i, row in enumerate(csv_reader):
                    
                    if len(row) == len(columns):
                        driver_ratings = None
                        if row[17].lower() not in ('null', ''):
                            try:
                                driver_ratings = float(row[17])
                            except ValueError:
                                pass 

                        customer_rating = None
                        if row[18].lower() not in ('null', ''):
                            try:
                                customer_rating = float(row[18])
                            except ValueError:                            
                                pass 

                        
                        booking_date = None
                        if row[0].lower() not in ('null', ''):
                            try: 
                                parsed_date = datetime.strptime(row[0], '%d-%m-%Y') 
                                booking_date = parsed_date.strftime('%Y-%m-%d')
                            except ValueError:
                                print(f"Warning: Could not parse date '{row[0]}' in row {i+2}. Setting to NULL.")
                                pass 

                        try:
                            val = (
                                booking_date, 
                                row[1], 
                                row[2], 
                                row[3], 
                                row[4], 
                                row[5], 
                                row[6], 
                                row[7], 
                                row[8], 
                                row[9], 
                                row[10],
                                row[11],
                                row[12],
                                row[13],
                                int(row[14]), 
                                row[15], 
                                int(row[16]), 
                                driver_ratings,
                                customer_rating
                            )
                            data_to_insert.append(val)
                            total_rows_processed += 1
                        except ValueError as ve:
                            print(f"Skipping row {i+2} due to data type conversion error (other than ratings/date): {ve} - Row: {row}")
                            continue 

                    else:
                        print(f"Skipping row {i+2} due to column mismatch (expected {len(columns)}, got {len(row)}): {row}")
                        continue

                    if len(data_to_insert) >= batch_size:
                        cursor.executemany(insert_query, data_to_insert)
                        conn.commit()
                        print(f"Inserted {total_rows_processed} rows so far...")
                        data_to_insert = [] 

                
                if data_to_insert:
                    cursor.executemany(insert_query, data_to_insert)
                    conn.commit()
                    print(f"Inserted final batch. Total rows inserted: {total_rows_processed}.")

                print(f"Data from '{csv_file_path}' successfully inserted into table '{table_name}'.")

    except Error as e:
        print(f"Error: {e}")
        if conn and conn.is_connected():
            conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn and conn.is_connected():
            conn.close()
            print("MySQL connection closed.")


if __name__ == "__main__":
    insert_data_from_csv(CSV_FILE, DATABASE_NAME, TABLE_NAME, DB_CONFIG)


Successfully connected to MySQL Database 'ola'!
Prepared INSERT query for table 'booking'.
Inserted 10000 rows so far...
Inserted 20000 rows so far...
Inserted 30000 rows so far...
Inserted 40000 rows so far...
Inserted 50000 rows so far...
Inserted 60000 rows so far...
Inserted 70000 rows so far...
Inserted 80000 rows so far...
Inserted 90000 rows so far...
Inserted 100000 rows so far...
Inserted final batch. Total rows inserted: 103024.
Data from 'Bookings-100000-Rows.csv' successfully inserted into table 'booking'.
MySQL connection closed.
